In [1]:
import re
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessagePromptTemplate
import os

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Objective: We are going to hand code a simple framework for the re-Act pattern.

This focuses on using a language model to act as an agent, that can use several tools, and follows the following pattern:

1. Observe the environment
2. Interpret the environment with a thought
3. Decide on an action
4. Act on the environment
5. Repeat steps 1 - 4 until we've find a solution or we've done too many iterations (the solution is "i've found an answer")


# How to extract the last action and action_input:


In [2]:
# Sample text
text = """
Action: search_on_google
Action_Input: Tom Hanks current wife

action: search_on_wikipedia
action_input: How old is Rita Wilson in 2023

action : search_on_google
action input: some other query
"""

# Compile regex patterns
action_pattern = re.compile(r"(?i)action\s*:\s*([^\n]+)", re.MULTILINE)
action_input_pattern = re.compile(r"(?i)action\s*_*input\s*:\s*([^\n]+)", re.MULTILINE)

# Find all occurrences of action and action_input
actions = action_pattern.findall(text)
action_inputs = action_input_pattern.findall(text)

# Extract the last occurrence of action and action_input
last_action = actions[-1] if actions else None
last_action_input = action_inputs[-1] if action_inputs else None

print("Last Action:", last_action)
print("Last Action Input:", last_action_input)

Last Action: search_on_google
Last Action Input: some other query


---

`action_pattern = re.compile(r"(?i)action\s*:\s*([^\n]+)", re.MULTILINE)`

`(?i)`: This is called an inline flag and makes the regex pattern case-insensitive. It means that the pattern will match "action", "Action", "ACTION", or any other combination of uppercase and lowercase letters.

`action`: This part of the pattern matches the word "action" literally. Due to the case-insensitive flag, it will match any capitalization of the word.

`\s*`: This part of the pattern matches zero or more whitespace characters (spaces, tabs, etc.). The \* means "zero or more" and \s is the regex shorthand for a whitespace character.

`:`: This part of the pattern matches the colon character literally.

`\s*`: This is the same as the previous \s\* part, matching zero or more whitespace characters after the colon.

`([^\n]+)`: This part of the pattern is a capturing group, denoted by the parentheses. It matches one or more characters that are NOT a newline character. The ^ inside the square brackets [] negates the character class, and \n represents the newline character. The + means "one or more". The text matched by this group will be extracted when using the findall() function.

`re.MULTILINE`: This is a flag passed to re.compile() function. It tells the regex engine that the input text may have multiple lines, so the pattern should be applied line by line.

In regular expressions, square brackets `[]` are used to define a character class, which is a set of characters that you want to match. For example, [abc] would match any single character that is either 'a', 'b', or 'c'.

When you add a caret `(^)` at the beginning of the character class, it negates the character class, meaning it will match any character that is NOT in the character class. In other words, it inverts the set of characters you want to match.

So, when we use `[^abc]`, it will match any single character that is NOT 'a', 'b', or 'c'. In the regex pattern `([^\n]+)`, the character class is `[^n]`, which means it will match any character that is NOT a newline character (\n). The + after the negated character class means that the pattern should match one or more characters that are not newlines.

By using the negated character class `[^n]` in the capturing group, we ensure that the regex engine captures text up to the end of the line without including the newline character itself. This is useful when we want to extract the text after the word "action" or "action input" up to the end of the line.

Overall, this regular expression pattern matches the word "action" (case-insensitive) followed by optional whitespace, a colon, optional whitespace again, and then captures any text up to the end of the line.


---


`action_input_pattern = re.compile(r"(?i)action_input\s*:\s*([^\n]+)", re.MULTILINE)`

The two regular expressions are very similar, with the only difference being the literal text they match at the beginning of each pattern. I'll briefly describe each regex and highlight the difference:

This pattern matches the word `"action_input"` (case-insensitive) followed by optional whitespace, a colon, optional whitespace again, and then captures any text up to the end of the line.

The only difference between these two regex patterns is the literal text they are looking for at the beginning:

action_pattern looks for the word `"action".`
action_input_pattern looks for the word `"action_input".`
Both patterns are case-insensitive, and they both capture the text following the matched word and the colon up to the end of the line. The purpose of these regex patterns is to extract the information after the keywords `"action"` and `"action_input"` from a given text.


---


In [3]:
def extract_last_action_and_input(text):
    # Compile regex patterns
    action_pattern = re.compile(r"(?i)action\s*:\s*([^\n]+)", re.MULTILINE)
    action_input_pattern = re.compile(
        r"(?i)action\s*_*input\s*:\s*([^\n]+)", re.MULTILINE
    )

    # Find all occurrences of action and action_input
    actions = action_pattern.findall(text)
    action_inputs = action_input_pattern.findall(text)

    # Extract the last occurrence of action and action_input
    last_action = actions[-1] if actions else None
    last_action_input = action_inputs[-1] if action_inputs else None

    return {"action": last_action, "action_input": last_action_input}

In [4]:
extract_last_action_and_input(text)

{'action': 'search_on_google', 'action_input': 'some other query'}

---


Given that we will also need to find out whether the language model has found the final answer, we will use the following template:

`"I've found the answer: final_answer"`


In [5]:
final_answer_text = "I've found the answer: final_answer"

# Write a regex to extract the final answer
final_answer_pattern = re.compile(
    r"(?i)I've found the answer:\s*([^\n]+)", re.MULTILINE
)

# Find all occurrences of the final answer
final_answers = final_answer_pattern.findall(final_answer_text)
print("Final Answers:", final_answers)

Final Answers: ['final_answer']


In [6]:
def extract_final_answer(text):
    final_answer_pattern = re.compile(
        r"(?i)I've found the answer:\s*([^\n]+)", re.MULTILINE
    )
    final_answers = final_answer_pattern.findall(text)
    if final_answers:
        return final_answers[0]
    else:
        return None

In [7]:
final_answer_text = "I've found the answer: final_answer"
print(extract_final_answer(final_answer_text))

final_answer


---


In [9]:
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

True

In [10]:
from langchain_openai.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessagePromptTemplate

chat = ChatOpenAI(
    model_kwargs={
        "stop": ["tool_result:"],
    }
)

tools = {}


def search_on_google(query: str):
    return f"Jason Derulo doesn't have a wife or partner."


tools["search_on_google"] = {
    "function": search_on_google,
    "description": "Searches on google for a query",
}


base_prompt = """
You will attempt to solve the problem of finding the answer to a question.
Use chain of thought reasoning to solve through the problem, using the following pattern:

1. Observe the original question:
original_question: original_problem_text
2. Create an observation with the following pattern:
observation: observation_text
3. Create a thought based on the observation with the following pattern:
thought: thought_text
4. Use tools to act on the thought with the following pattern:
action: tool_name
action_input: tool_input

Do not guess or assume the tool results. Instead, provide a structured output that includes the action and action_input.

You have access to the following tools: {tools}.

original_problem: {question}
"""

model_output = chat.invoke(
    SystemMessagePromptTemplate.from_template(template=base_prompt).format_messages(
        tools=tools, question="Is Jason Derulo with a partner?"
    )
)
print(model_output)


# Extract the tool_name and tool_input from the model_output
tool_name = extract_last_action_and_input(model_output.content)["action"]
tool_input = extract_last_action_and_input(model_output.content)["action_input"]
tool_result = tools[tool_name]["function"](tool_input)

print(
    f"""
----------
The agent has opted to use the following tool:
tool_name: {tool_name}
tool_input: {tool_input}
tool_result: {tool_result}
----------
"""
)

current_prompt = """
Based on the provided tool result:
tool_result: {tool_result}

Either provide the next observation, action, action_input, or the final answer if available.
If you are providing the final answer, you must return the following pattern:
"I've found the answer: final_answer" """

print("The second prompt shows", current_prompt)

model_output = chat(
    SystemMessagePromptTemplate.from_template(template=current_prompt).format_messages(
        tool_result=tool_result
    )
)

print("----------\n\nThe model output is:", model_output.content)
# See if there is a final answer:
final_answer = extract_final_answer(model_output.content)
if final_answer:
    print(f"answer: {final_answer}")
else:
    print("No final answer found.")

/Users/gordontveito-duncan/Documents/GitHub/prompt-engineering-for-generative-ai-examples/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3490: UserWarning: Parameters {'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


content="1. Observe the original question:\noriginal_question: Is Jason Derulo with a partner?\n\n2. Create an observation:\nobservation: Jason Derulo's relationship status is not always publicly known, so we might need to look up the latest information to find out if he is currently with a partner.\n\n3. Create a thought based on the observation:\nthought: We can search online to find the most recent information about Jason Derulo's relationship status.\n\n4. Use a tool to act on the thought:\naction: search_on_google\naction_input: Jason Derulo current relationship status" additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 116, 'prompt_tokens': 192, 'total_tokens': 308, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'f

/var/folders/0k/3rxx7x4x7_7f028m32kgbqzh0000gn/T/ipykernel_68692/4196085321.py:78: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  model_output = chat(


----------

The model output is: Do you want to know more information about Jason Derulo's personal life or career?
No final answer found.
